# Labo 7 : classification de textes (pour la désambiguïsation lexicale)
Étudiant·e 1 & Étudiant·e 2

## Objectif et plan

L’objectif de ce laboratoire est d’utiliser des méthodes d’apprentissage supervisé pour classifier des occurrences du mot interest selon leur sens : c’est la même tâche, avec les mêmes données, que le labo 6. Pour classifier les occurrences, on considère leur contexte (mots voisins), et on applique l’approche bayésienne vue en cours : on entraîne un classifieur à six classes (les six sens de interest annotés de 1 à 6) sur une partie des données et on le teste sur la partie restante.

Dans ce laboratoire, on explore deux façons différentes de coder les traits (features) pour cette tâche. Dans les deux cas, on entraînera un NaiveBayesClassifier fourni par NLTK.1 Les deux façons sont :
1. Constituer un vocabulaire des mots qui apparaissent dans le voisinage de interest et définir ces mots comme traits. Pour chaque occurrence de interest, on extrait la valeur de ces traits sous la forme {(‘rate’ : True), (‘in’ : False), … } et on ajoute la classe (de 1 à 6).
2. Si word-1 est le mot précédant l’occurrence de interest, on définit comme traits word-n, …, word-2, word-1, word+1, word+2, …, word+n (une fenêtre de taille 2n autour de interest). Les valeurs possibles de ces traits sont cette fois-ci les mots observés, ou ‘NONE’ si la fenêtre dépasse les limites de la phrase. Pour chaque occurrence de interest, on extrait la valeur de ces traits sous la forme {(‘word-1’ : ‘his’), (‘word+1’ : ‘in’), … } et on ajoute la classe (de 1 à 6).

Dans les deux cas, il faut diviser les 2368 occurrences de interest en un jeu d’entraînement et un jeu de test, en respectant la proportion initiale de chaque sens. Puis on entraîne un classifieur bayésien naïf en respectant le format de données indiqué par NLTK2, et on teste la performance du classifieur entraîné. L’objectif est de trouver les paramètres qui conduisent aux meilleurs scores de WSD.

## Étapes proposées

### A. Traits lexicaux : présence ou absence de mots dans le voisinage de interest

#### 1. Le fichier de données se trouve à http://www.d.umn.edu/~tpederse/data.html – chercher « interest » vers la fin de la page, et prendre le fichier marqué comme « original format without POS tags » (le même qu’au labo 6). Lire le fichier et générer une liste de listes de mots (une liste par phrase) appelée tokenized_sentences.

In [2]:
import nltk
# Lecture du document contenant les occurences d' 'interest'
f = open("data/interest-original.txt", "r").read()
sent_brut = nltk.sent_tokenize(f)

# Nettoyage des phrases
tokenized_sentences = [nltk.word_tokenize(s.strip("$\n= ").replace("=", "").replace("  ", " ")) for s in sent_brut ]
tokenized_sentences

[['yields',
  'on',
  'money-market',
  'mutual',
  'funds',
  'continued',
  'to',
  'slide',
  ',',
  'amid',
  'signs',
  'that',
  'portfolio',
  'managers',
  'expect',
  'further',
  'declines',
  'in',
  'interest_6',
  'rates',
  '.'],
 ['longer',
  'maturities',
  'are',
  'thought',
  'to',
  'indicate',
  'declining',
  'interest_6',
  'rates',
  'because',
  'they',
  'permit',
  'portfolio',
  'managers',
  'to',
  'retain',
  'relatively',
  'higher',
  'rates',
  'for',
  'a',
  'longer',
  'period',
  '.'],
 ['nevertheless',
  ',',
  'said',
  'brenda',
  'malizia',
  'negus',
  ',',
  'editor',
  'of',
  'money',
  'fund',
  'report',
  ',',
  'yields',
  '``',
  'may',
  'blip',
  'up',
  'again',
  'before',
  'they',
  'blip',
  'down',
  '``',
  'because',
  'of',
  'recent',
  'rises',
  'in',
  'short-term',
  'interest_6',
  'rates',
  '.'],
 ['j.p.',
  'bolduc',
  ',',
  'vice',
  'chairman',
  'of',
  'w.r.',
  'grace',
  '&',
  'co.',
  ',',
  'which',
  'hol

#### 2. Définir une variable window_size, par exemple égale à 3 (on la fera varier plus tard), et une liste vide de mots word_list. Parcourir les tokenized_sentences et pour chaque phrase ajouter les mots voisins de interest (i.e. situés à une distance inférieure ou égale à window_size) dans la liste de mots word_list. Combien de mots contient celle-ci à la fin ? Tokens ou types ?

In [3]:
window_size = 3
word_list = []
for sentence in tokenized_sentences:
    for word in sentence:
        if 'interest' in word:
            idx = sentence.index(word)
            sentence[idx-window_size:idx+window_size+1]
            word_list.extend(sentence[idx-window_size:idx])
            word_list.extend(sentence[idx+1:idx+window_size+1])
print("La liste de mots contient " + str(len(word_list)) + " tokens.")

La liste de mots contient 14059 tokens.


#### 3. À l’aide d’un objet NLTK de type FreqDist, sélectionner parmi les mots de word_list les N plus fréquents, dans une nouvelle liste appelée vocabulary (p.ex. N = 500, mais on le fera varier). Affichez les 50 mots les plus fréquents. Est-ce une bonne idée d’enlever les stopwords ?

In [4]:
fdist = nltk.FreqDist(word_list)
vocabulary = fdist.most_common(500)
vocabulary[:50]

[('in', 808),
 ('rates', 638),
 ('the', 636),
 (',', 537),
 ('and', 415),
 ('.', 381),
 ('to', 377),
 ('of', 351),
 ('a', 271),
 ('on', 178),
 ('rate', 149),
 ('its', 148),
 ('%', 147),
 ("'s", 138),
 ('payments', 123),
 ('that', 104),
 ('are', 102),
 ('``', 96),
 ('has', 91),
 ('for', 88),
 ('lower', 83),
 ('with', 82),
 ('an', 82),
 ('by', 79),
 ('is', 76),
 ('have', 70),
 ('will', 66),
 ('high', 66),
 ('at', 62),
 ('from', 58),
 ('or', 52),
 ('short', 51),
 ('annual', 50),
 ('their', 48),
 ('company', 47),
 ('which', 47),
 ('minority', 46),
 ("n't", 45),
 ('foreign', 45),
 ('bonds', 44),
 ('u.s.', 43),
 ('as', 43),
 ('income', 43),
 ('higher', 43),
 ('other', 40),
 ('pay', 39),
 ('it', 38),
 ('below', 35),
 ('because', 34),
 ('buying', 34)]

#### 4. Parcourir à nouveau les tokenized_sentences et pour chaque phrase créer un couple (dictionnaire, sens), où le dictionnaire regroupe les traits et leurs valeurs, et le sens est un nombre de 1 à 6 indiquant le sens de interest. Les couples pour toutes les phrases seront rassemblés dans une liste appelée feature_sets.

##### 4.1 Prendre modèle sur https://www.nltk.org/book/ch06.html (début du 1.2)

##### 4.2 Pour le dictionnaire, il faut créer un trait pour chaque mot de vocabulary, et examiner si ce mot est présent dans une fenêtre de taille window_size autour de l’occurrence de interest : si oui, le trait est True, sinon il est False. Par exemple, on aboutit à : {'contains(the)': False, 'contains(,)': True, 'contains(rates)': True, …}.

In [87]:

def dicoValueGen(sentences, voc):
    f_sets = []
    currentSet = []
    for sentence in sentences:
        dico = {}
        for word in sentence:
            if 'interest' in word:
                idx = sentence.index(word)
                currentSet = (sentence[idx-window_size:idx+window_size+1])
                if '_' in word:
                    sense = word[-1]
                dico['word0'] = 'interests' in word
            for v, occurence in voc:
                dico[v] = v in currentSet
        f_sets.append((dico, sense))
    return(f_sets)
    


In [89]:
feature_sets = dicoValueGen(tokenized_sentences, vocabulary)

##### 4.3 Ajouter aussi le trait ‘word0’ qui note si l’occurrence est interest ou interests (pluriel).

##### 4.3 Combien d’occurrences pour chaque sens de interest y a-t-il dans feature_sets ?

In [90]:
from collections import Counter
Counter(e[1] for e in feature_sets)

Counter({'1': 364, '2': 11, '3': 66, '4': 181, '5': 508, '6': 1260})

#### 5. Diviser les données de feature_sets en deux sous-ensembles : l’un comportant 80% des données est le train_set, et l’autre (20%) est le test_set. Attention, il faut respecter deux conditions :

##### 5.1 Chaque sens doit être présent dans les mêmes proportions dans train_set et dans test_set (donc il faut faire la division de manière séparée pour chaque sens).

In [91]:
from collections import defaultdict
groupedSenses = defaultdict(list)

for dico, sense in feature_sets:
    groupedSenses[sense].append((dico, sense))
groupedSenses['1'][0]

({'$': False,
  '%': False,
  '&': False,
  "'": False,
  "'s": False,
  '(': False,
  ')': False,
  '*interest': False,
  ',': False,
  '--': False,
  '.': False,
  '1': False,
  '10': False,
  '12': False,
  '14': False,
  '15': False,
  '1987': False,
  '2': False,
  '20': False,
  '25': False,
  '32': False,
  '45': False,
  '45.67': False,
  '50': False,
  '6.25': False,
  '60': False,
  ':': False,
  ';': False,
  '``': False,
  'a': False,
  'about': False,
  'accrued': False,
  'accruing': False,
  'acquire': False,
  'acquired': False,
  'acquiring': False,
  'additional': False,
  'advantage': False,
  'after': False,
  'again': False,
  'against': False,
  'all': False,
  'allow': False,
  'allowing': False,
  'already': False,
  'also': False,
  'american': False,
  'among': False,
  'amount': False,
  'an': False,
  'and': False,
  'annual': False,
  'any': False,
  'are': False,
  'as': False,
  'at': False,
  'attracted': False,
  'australian': False,
  'back': False,
  

##### 5.2 Mélanger avec shuffle() les occurrences avant de prendre les 80% premières pour le `train_set`et les 20% restantes dans le test_set.

In [92]:
import random
import math
def randomSet(data):
    set1 = []
    set2 = []
    for sense in data:
        trainLen = math.ceil(len(data[sense])*0.8)
        random.shuffle(data[sense])
        set1.extend(data[sense][:trainLen])
        set2.extend(data[sense][trainLen:])
    return(set1,set2)

train_set, test_set = randomSet(groupedSenses)

random.shuffle(train_set)
random.shuffle(test_set)

#### 6. Entraîner un classifieur de type NaiveBayesClassifier de NLTK sur train_set, puis le tester sur les données de test_set. Quelle est la précision (accuracy) atteinte ?

In [93]:
import nltk
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [94]:
nltk.classify.accuracy(classifier, test_set)

0.8613445378151261

#### 7. Adapter le code précédent pour effectuer plusieurs divisions des données en train et test (par exemple 10), et calculer la moyenne des scores obtenus. Comment se compare cette moyenne avec votre premier résultat ?

In [95]:
scores = []
for i in range(0, 10):
    train_set, test_set = randomSet(groupedSenses)
    random.shuffle(train_set)
    random.shuffle(test_set)
    classifier = nltk.NaiveBayesClassifier.train(train_set)
    scores.append(nltk.classify.accuracy(classifier, test_set))
sum(scores)/10

0.8508403361344536

#### 8. Cherchez les meilleurs paramètres pour la taille de la fenêtre (p.ex. 1, 3, 5, 7, 11) et la taille du vocabulaire (50, 100, 200, 500, 1000 mots). Combien d’expériences faut-il exécuter ? Quelle est la meilleure combinaison fenêtre x vocabulaire et quel est le score moyen obtenu ?

### B. Traits lexicaux positionnels : valeurs des mots précédant/suivant interest
Pour cette deuxième partie, on réutilisera beaucoup d’éléments de la première. Seule la nature des
traits utilisés et leur extraction vont changer.

#### 1. Partir de la liste de listes de mots (une liste par phrase) précédente, appelée tokenized_sentences.

In [74]:
tokenized_sentences

[['yields',
  'on',
  'money-market',
  'mutual',
  'funds',
  'continued',
  'to',
  'slide',
  ',',
  'amid',
  'signs',
  'that',
  'portfolio',
  'managers',
  'expect',
  'further',
  'declines',
  'in',
  'interest_6',
  'rates',
  '.'],
 ['longer',
  'maturities',
  'are',
  'thought',
  'to',
  'indicate',
  'declining',
  'interest_6',
  'rates',
  'because',
  'they',
  'permit',
  'portfolio',
  'managers',
  'to',
  'retain',
  'relatively',
  'higher',
  'rates',
  'for',
  'a',
  'longer',
  'period',
  '.'],
 ['nevertheless',
  ',',
  'said',
  'brenda',
  'malizia',
  'negus',
  ',',
  'editor',
  'of',
  'money',
  'fund',
  'report',
  ',',
  'yields',
  '``',
  'may',
  'blip',
  'up',
  'again',
  'before',
  'they',
  'blip',
  'down',
  '``',
  'because',
  'of',
  'recent',
  'rises',
  'in',
  'short-term',
  'interest_6',
  'rates',
  '.'],
 ['j.p.',
  'bolduc',
  ',',
  'vice',
  'chairman',
  'of',
  'w.r.',
  'grace',
  '&',
  'co.',
  ',',
  'which',
  'hol

#### 2. Définir une variable window_size2, par exemple égale à 3 (on la fera varier plus tard).

In [75]:
window_size2 = 3

#### 3. Parcourir les tokenized_sentences et pour chaque phrase créer un couple (dictionnaire, sens), où le dictionnaire regroupe les traits et leurs valeurs, et le sens est un nombre de 1 à 6 indiquant le sens de interest. Les couples pour toutes les phrases seront rassemblés dans une nouvelle liste appelée feature_sets2.

##### 3.1 Pour le dictionnaire de traits, il faut cette fois-ci créer un trait pour chaque position relative par rapport à interest, donc ‘word-1’, ‘word+1’, etc. (jusqu’à window_size2). La valeur du trait sera le mot trouvé à cette position, ou ‘NONE’ si on sort de la phrase. Par exemple {(‘word-1’ : ‘his’), (‘word+1’ : ‘in’), … }.

##### 3.2 Ajouter aussi le trait ‘word0’ qui note si l’occurrence est interest ou interests (pluriel).

#### 4. Diviser les données de feature_sets2 en deux sous-ensembles (80%/20%) appelés train_set2 et test_set2 avec la même procédure qu’à la partie A.

#### 5. Entraîner un classifieur de type NaiveBayesClassifier de NLTK sur train_set2, puis le tester sur les données de test_set2. Quelle est la précision (accuracy) atteinte ?

#### 6. Effectuer plusieurs divisions des données en train et test (par exemple 10), et calculer la moyenne des scores obtenus. Comment se compare cette moyenne avec votre premier résultat ?

#### 7. Cherchez les meilleurs paramètres pour la taille de la fenêtre (p.ex. entre 1 et 15). Quelle est la meilleure valeur et quel est le score moyen obtenu ?

#### 8. Quelle est le meilleur score obtenu entre (A) et (B) ?

#### 9. Thème de réflexion facultatif : les différences des scores sont-elles statistiquement significatives ?

Merci d’envoyer votre notebook Jupyter par email au professeur avant le **lundi 27 mai à 23h59**.